In [20]:
from tensorflow import keras
from tensorflow.keras import layers

In [21]:
inputs = keras.Input(shape=(3,), name = "my_input")
features = layers.Dense(64, activation = "relu", name = "layer_1")(inputs)
outputs = layers.Dense(10, activation = "sigmoid", name = "final_layer")(features)

model = keras.Model(inputs = inputs, outputs = outputs)

In [22]:
inputs.shape

(None, 3)

In [23]:
inputs.dtype

'float32'

In [24]:
features.shape

(None, 64)

In [25]:
features.dtype

'float32'

In [26]:
outputs.shape

(None, 10)

In [27]:
outputs.dtype

'float32'

In [28]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ my_input (InputLayer)           │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_1 (Dense)                 │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ final_layer (Dense)             │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 906 (3.54 KB)

 Trainable params: 906 (3.54 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Multi-input, Multi-output models for customer support tickets ranking by priority and then routing them to appropriate department

In [30]:
# Input: title of ticket (text), body of ticket (text), and tags added (categorical)
# encode text inputs as arrays of ones and zeros of size vocabulary_size

# Output: priority score of ticket (sigmoid), department (softmax over the set of departments)

In [31]:
vocabulary_size = 10000
num_tags = 100

num_departments = 4

In [38]:
# model inputs
title = keras.Input(shape=(vocabulary_size,), name = "title")
print(title.shape) # (None, 10000) where None = batch_size
text_body = keras.Input(shape=(vocabulary_size, ), name = "text_body")
tags = keras.Input(shape=(num_tags, ), name = "tags")
print(tags.shape) # (None, 100)

(None, 10000)
(None, 100)


In [39]:
features = layers.Concatenate()([title, text_body, tags])
print(features.shape) # (None, 20100)

features = layers.Dense(64, activation = "relu")(features)
print(features.shape) # (None, 64)

(None, 20100)
(None, 64)


In [69]:
# model outputs
priority = layers.Dense(1, activation = "sigmoid", name = "priority")(features)
print(priority.shape) # (None, 1)

department = layers.Dense(num_departments, activation = "softmax", name = "department") (features)
print(department.shape) # (None, 4)

(None, 1)
(None, 4)


In [70]:
model = keras.Model(inputs = [title, text_body, tags],
                    outputs = [priority, department])
print(model.summary())

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ title (InputLayer)  │ (None, 10000)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_body           │ (None, 10000)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tags (InputLayer)   │ (None, 100)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 20100)     │          0 │ title[0][0],      │
│ (Concatenate)       │                   │            │ text_body[0][0],  │
│                     │                   │            │ tags[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │  1,286,464 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ priority (Dense)    │ (None, 1)         │         65 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department (Dense)  │ (None, 4)         │        260 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,286,789 (4.91 MB)

 Trainable params: 1,286,789 (4.91 MB)

 Non-trainable params: 0 (0.00 B)

None


In [71]:
# training a multi-input, multi-output model

In [72]:
import numpy as np

num_samples = 1280

# dummy input data
title_data = np.random.randint(0, 2, size = (num_samples, vocabulary_size))
print(title_data.shape) # (1280, 10000)

text_body_data = np.random.randint(0, 2, size = (num_samples, vocabulary_size))
print(text_body_data.shape) # (1280, 10000)


tags_data = np.random.randint(0, 2, size = (num_samples, num_tags))
print(tags_data.shape) # (1280, 100)

# dummy target data
priority_data = np.random.random(size=(num_samples, 1))
print(priority_data.shape) # (1280, 1)

department_data = np.random.randint(0, 2, size = (num_samples, num_departments))
print(department_data.shape) # (1280, 4)


model.compile(optimizer = "rmsprop",
              loss = ["mean_squared_error", "categorical_crossentropy"],
              metrics = [["mean_absolute_error"], ["accuracy"]])

model.fit([title_data, text_body_data, tags_data],
          [priority_data, department_data], epochs = 1)

model.evaluate([title_data, text_body_data, tags_data], [priority_data, department_data])

priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])
print(priority_preds.shape) # (1280, 1)

print(department_preds.shape) # (1280, 4)

(1280, 10000)
(1280, 10000)
(1280, 100)
(1280, 1)
(1280, 4)
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - department_accuracy: 0.2297 - department_loss: 2063.6921 - loss: 2064.0339 - priority_loss: 0.3416 - priority_mean_absolute_error: 0.5051
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - department_accuracy: 0.2584 - department_loss: 380.3897 - loss: 380.7254 - priority_loss: 0.3357 - priority_mean_absolute_error: 0.5001
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
(1280, 1)
(1280, 4)


In [73]:
print(priority_preds[0])

[0.]


In [74]:
print(department_preds[0])

[2.1739386e-34 9.9999994e-01 2.1214802e-21 0.0000000e+00]


In [75]:
print(np.argmax(department_preds[0]))

1


In [76]:
# training a model by providing dicts of input and target arrays

model.compile(optimizer = "rmsprop",
              loss = {"priority": "mean_squared_error",
                      "department": "categorical_crossentropy"},
              metrics = {"priority": ["mean_absolute_error"],
                         "department": ["accuracy"]})

model.fit({"title": title_data, "text_body": text_body_data, "tags": tags_data},
          {"priority": priority_data, "department": department_data}, epochs = 1)

model.evaluate({"title": title_data, "text_body": text_body_data, "tags": tags_data},
               {"priority": priority_data, "department": department_data})

priority_preds, department_preds = model.predict({"title": title_data, "text_body": text_body_data, "tags": tags_data})

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - department_accuracy: 0.2913 - department_loss: 201.3344 - loss: 201.6762 - priority_loss: 0.3419 - priority_mean_absolute_error: 0.5077
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - department_accuracy: 0.1249 - department_loss: 65.3107 - loss: 65.6464 - priority_loss: 0.3357 - priority_mean_absolute_error: 0.5001
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [77]:
print(priority_preds[0])

[0.]


In [78]:
print(department_preds[0])

[1.5047607e-38 1.0214944e-08 9.9999994e-01 1.2780473e-10]


In [79]:
print(np.argmax(department_preds[0]))

2
